# **6th place solution : Tabular Feb. Competition**

It is my first time to reach this high ranking. So I am really happy!
This is my code I used to improve my score. 
I refer to lots of discussion and codes, and It was really helpful. Thanks you for everybody who wrote it. 
And special thanks to @hiro5299834! Wrote amazing LGBM codes and fine-tuned params, it became our code baseline. [His Code](https://www.kaggle.com/hiro5299834/tps-feb-2021-with-single-lgbm-tuned)


# **Abstract**
I used pseudo labelling for improving score. Pseudo Labelling is kind of semi-supervised learning. I labelled all test data, and trained again with original train data and pseudo-labelled test data. [Detail](https://analyticsindiamag.com/pseudo-labelling-a-guide-to-semi-supervised-learning/) 
It is really simple trick, but score improved a lot. Original code was 0.84198 LB, but pseudo labelling code reached 0.84185 LB (0.84249 private score) 

After this, I ensembled a lot. And I can reached 0.84246 private score. But I couldn't see the public score improved, so I couldn't select that for final submission, unfortunately. 

Below is my code for training 6th place model.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import sys
import math
import time
import pickle
import psutil
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from contextlib import contextmanager

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.manifold import TSNE

import warnings

import optuna
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
from sklearn.preprocessing import (StandardScaler, PowerTransformer, MinMaxScaler,
                                   QuantileTransformer, LabelEncoder,
                                   OneHotEncoder, OrdinalEncoder)
import category_encoders as ce
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')

# **Pseudo Labelling**
Just concat well-trained submission file, test file, and train file.

In [ ]:
original_train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
test_target = pd.read_csv('../input/tabular-feb-submission-files/sub45_47_48_(5).csv')

In [ ]:
test['target'] = test_target.target

In [ ]:
train = pd.concat([original_train, test], axis=0)
train

# **Training with LGBM**

This code is from BIZEN's codes, but pseudo labelling always worked with any LGBM models. So you can use any fine-tuned LGBM models. Actually, I reached 0.84245 private score with single LGBM model and pseudo labelling. (0.84193 public LB) 

In [ ]:
N_FOLDS = 10
N_ESTIMATORS = 30000
SEED = 2021
BAGGING_SEED = 48

In [ ]:
@contextmanager
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)


def score_log(df: pd.DataFrame, seed: int, num_fold: int, model_name: str, cv: float):
    score_dict = {'date': datetime.now(), 'seed': seed, 'fold': num_fold, 'model': model_name, 'cv': cv}
    # noinspection PyTypeChecker
    df = pd.concat([df, pd.DataFrame.from_dict([score_dict])])
    df.to_csv(LOG_PATH / f"model_score_{model_name}.csv", index=False)
    return df

In [ ]:
with timer("Read data"):
    train_df = train
    test_df = pd.read_csv("../input/tabular-playground-series-feb-2021/test.csv")
    sub_df = pd.read_csv( "../input/tabular-playground-series-feb-2021/sample_submission.csv")

    all_df = pd.concat([train_df, test_df])
    all_df = all_df.sort_values('id').reset_index(drop=True)

In [ ]:
cont_features = [f'cont{i}' for i in range(14)]
cat_features = [f'cat{i}' for i in range(10)]
all_features = cat_features + cont_features
target_feature = 'target'

target = train_df[target_feature]
train_df = train_df[all_features]
test_df = test_df[all_features]

In [ ]:
# LabelEncoder()
with timer("LabelEncoder"):
    le = LabelEncoder()
    for col in cat_features:
        le.fit(train_df[col])
        train_df[col] = le.transform(train_df[col])
        test_df[col] = le.transform(test_df[col])

In [ ]:
params = {'random_state': SEED,
          'metric': 'rmse',
          'n_estimators': N_ESTIMATORS,
          'n_jobs': -1,
          'cat_feature': [x for x in range(len(cat_features))],
          'bagging_seed': SEED,
          'feature_fraction_seed': SEED,
          'learning_rate': 0.003899156646724397,
          'max_depth': 99,
          'num_leaves': 63,
          'reg_alpha': 9.562925363678952,
          'reg_lambda': 9.355810045480153,
          'colsample_bytree': 0.2256038826485174,
          'min_child_samples': 290,
          'subsample_freq': 1,
          'subsample': 0.8805303688019942,
          'max_bin': 882,
          'min_data_per_group': 127,
          'cat_smooth': 96,
          'cat_l2': 19
          }

In [ ]:
oof = np.zeros(train_df.shape[0])
preds = 0
train_preds = 0
score_df = pd.DataFrame()
feature_importances = pd.DataFrame()

kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

In [ ]:
for fold, (train_idx, valid_idx) in enumerate(kf.split(X=train_df)):
    X_train, X_valid = train_df.iloc[train_idx], train_df.iloc[valid_idx]
    y_train, y_valid = target.iloc[train_idx], target.iloc[valid_idx]

    with timer(f"fold {fold}: fit"):
        model = LGBMRegressor(**params)
        model.fit(X_train, y_train,
                  eval_set=[(X_valid, y_valid)],
                  eval_metric='rmse',
                  early_stopping_rounds=100,
                  verbose=0)

    fi_tmp = pd.DataFrame()
    fi_tmp['feature'] = model.feature_name_
    fi_tmp['importance'] = model.feature_importances_
    fi_tmp['fold'] = fold
    fi_tmp['seed'] = SEED
    feature_importances = feature_importances.append(fi_tmp)

    oof[valid_idx] = model.predict(X_valid)
    preds += model.predict(test_df)/N_FOLDS
    train_preds += model.predict(train_df)/N_FOLDS
    rmse = mean_squared_error(y_valid, oof[valid_idx], squared=False)
    score_df = score_log(score_df, SEED, fold, 'lgb', rmse)
    print(f"rmse {rmse}")

rmse = mean_squared_error(target, oof, squared=False)
score_df = score_log(score_df, SEED, 999, 'lgb', rmse)
print("+-"*40)
print(f"rmse {rmse}")

In [ ]:
pred_train_df = pd.DataFrame([])
sub_df.target = preds
sub_df.to_csv(f"pseudo_labelling_cv{rmse:.6f}.csv", index=False)
sub_df.head()

pred_train_df.target = train_preds
pred_train_df.to_csv("train_pred_v0.csv", index=False)

np.save(LOG_PATH / 'train_oof', oof)
np.save(LOG_PATH / 'test_preds', preds)

**Please upvote it was helpful for you! Thank you!!!**